In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from imagingIO import loopMice, loadData, loadBeh, getBeh
from analysis import dffCalc, filtData, smoothData
from events import getEvents
from trials import markTrials
from bouts import getBouts, getBoutDur, getBouts_
from statistics import stdCalc, getBoutMeans, getStats, getBoutMeans_
from plotter import plotMean, plotHeat, plotTrials
from scipy.signal import resample

In [2]:
# Start gathering the data from FR1 exp
# Mice
mice = [(8404,6), (8857, 3), (8864, 1)]
# Parameters
behType = 'FR1'
fs = 0.05
trials=[1,12]

In [3]:
# Get file paths
fileList = loopMice(mice, behType)

# Load the data
dataList = loadData(mice, behType)

# Load the behaviour
eventList = getBeh(mice, fileList['Behaviour'], behType)

# FIlter and dFF
filtList = filtData(mice, dffCalc(mice, dataList, lowest=False), cutoff=3.0, order=6)
dFFList = smoothData(mice, filtList, window=5)

In [81]:
# Eating bout
Start = getEvents(mice, eventList, 'Eat_Start', trials)
End = getEvents(mice, eventList, 'Eat_End', trials)
# Baseline equal to the event duration
base = Start-End
#
eventStart = Start - base
eventEnd = End + base
baseEvent = Start

boutsData =  getBouts_(mice, dFFList, eventStart, eventEnd, baseEvent, base, behType, trials, baselining=True)


3  mice were loaded.

Baseline was set -8.27777777778 sec before the event.


In [102]:
# Eating bout
Start = getEvents(mice, eventList, 'Eat_Start', trials)
End = getEvents(mice, eventList, 'Eat_End', trials)
# Baseline equal to the event duration
base = Start-End
#
eventStart = Start
eventEnd = End
baseEvent = Start

boutsData =  getBouts_(mice, dFFList, eventStart, eventEnd, baseEvent, base, behType, trials, baselining=True)


3  mice were loaded.

Baseline was set -8.27777777778 sec before the event.


In [121]:
samples = 500
cellNames = boutsData['Cell'].drop_duplicates().values
resampledData = pd.DataFrame()
for i, cell in enumerate(cellNames):
    print "\r", str(np.round(i*100./(len(cellNames)-1),1)), "%",
    for trial in np.arange(trials[0], trials[1]+1):
        trace = boutsData.pivot_table(index=['Cell', 'New_Time'], columns='Event', values='Fluoro').T[cell].loc[trial].dropna()
        resampledTrace, resampledTime = resample(trace, samples, t=trace.index)
        resampled = pd.DataFrame({'Resampled' : resampledTrace})
        resampled['New_Time'] = resampledTime
        resampled['Event'] = trial
        resampled['Cell'] = cell
        resampledData = resampledData.append(resampled)

100.0 %


In [117]:
samples = 500
cellNames = boutsData['Cell'].drop_duplicates().values
cell=cellNames[2]
trial=1
trace = boutsData.pivot_table(index=['Cell', 'New_Time'], columns='Event', values='Fluoro').T[cell].loc[trial].dropna()
resample(trace, samples, t=trace.index)
resampledTrace, resampledTime = resample(trace, samples, t=trace.index)

In [120]:
resampledData

""


In [118]:
r=resampledData.pivot_table(index=['Cell', 'Event'], columns='New_Time', values='Resampled').values.mean(axis=0)
t=resampledData.pivot_table(index=['Cell', 'Event'], columns='New_Time', values='Resampled').columns.values
plt.plot(t, r)
len(r), len(t)

KeyError: 'Cell'

In [ ]:
# Eating bout
Start = getEvents(mice, eventList, 'Eat_Start', trials)
End = getEvents(mice, eventList, 'Eat_End', trials)
# Baseline equal to the event duration
base = Start-End
#
eventStart = Start-base
eventEnd = End+base
baseEvent = Start

boutsData =  getBouts_(mice, dFFList, eventStart, eventEnd, baseEvent, base, behType, trials, baselining=True)

In [ ]:
samples = 500
cellNames = boutsData['Cell'].drop_duplicates().values
resampledData = pd.DataFrame()
for i, cell in enumerate(cellNames):
    print "\r", str(np.round(i*100./(len(cellNames)-1),1)), "%",
    for trial in np.arange(trials[0], trials[1]+1):
        trace = boutsData.pivot_table(index=['Cell', 'New_Time'], columns='Event', values='Fluoro').T[cell].loc[trial].dropna()
        resampled = pd.DataFrame({'Resampled' : resample(trace, samples)})
        resampled['New_Time'] = np.linspace(-1.0, 2.0, samples)
        resampled['Event'] = trial
        resampled['Cell'] = cell
        resampledData = resampledData.append(resampled)

In [ ]:
r=resampledData.pivot_table(index=['Cell', 'Event'], columns='New_Time', values='Resampled').values.mean(axis=0)
t=resampledData.pivot_table(index=['Cell', 'Event'], columns='New_Time', values='Resampled').columns.values
plt.plot(t, r)
len(r), len(t)